In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import urllib.request as urllib
import io
import zipfile
import re
import time
import datetime 
import dynetx as dn
import copy
import ast
import glob
import scipy
import community as community_louvain
import matplotlib.cm as cm

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from tensorflow import keras
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from rdyn import RDyn
from datetime import datetime
from sklearn.decomposition import PCA

from helpers import *
from Autoencoder import *

In [2]:
t = 5
n = 500
l = 256
epochs = 1000
k = 7

In [3]:
#graphs = pickle.load(open("synfix.p", "rb"))

graphs = []
for x in range(t):
    graph = pickle.load(open("./graphs/DANCer_12comm/t" + str(x) + ".p", "rb"))
    graphs.append(graph)

In [4]:
matrices = []
for graph in graphs:
    matrices.append(tf.convert_to_tensor(adjacency_to_similarity(create_adjacency_matrix(graph)).astype('float32')))

In [5]:
#static CD
static_aes = []
static_latent_spaces = []
for x in range(t):
    temp = Autoencoder(n, l, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
    B = matrices[x]
    history_temp = train(temp, epochs, n, B)
    H = temp.encoder(B)
    static_aes.append((temp, history_temp))
    static_latent_spaces.append(H)

In [6]:
#dynamic CD
dynamic_aes = []
dynamic_latent_spaces = []
last_H = static_latent_spaces[0]
dynamic_latent_spaces.append(last_H)
for x in range(1, t):
    temp = Autoencoder(n, l, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer(), subspace_distance=1)
    B = matrices[x]
    temp.set_past_embedding(last_H)
    history_temp = train(temp, epochs, n, B)
    H = temp.encoder(B)
    dynamic_aes.append((temp, history_temp))
    dynamic_latent_spaces.append(H)
    last_H = H

In [7]:
#create static labelings
static_labels = []
for x in range(t):
    H = static_latent_spaces[x]
    kmeans = KMeans(n_clusters=k, n_init=20).fit(H)
    static_labels.append(kmeans.labels_)

In [8]:
#crerate dynamic labelings
dynamic_labels = []
for x in range(t):
    H = dynamic_latent_spaces[x]
    kmeans = KMeans(n_clusters=k, n_init=20).fit(H)
    dynamic_labels.append(kmeans.labels_)

In [9]:
#if graphs have ground truths, can compute NMIs
static_NMIs = []
dynamic_NMIs = []

x = 0
for x in range(t):
    static_results = static_labels[x]
    dynamic_results = dynamic_labels[x]
    ground_truth = graph_labels(graphs[x])
    static_NMIs.append(normalized_mutual_info_score(static_results, ground_truth))
    dynamic_NMIs.append(normalized_mutual_info_score(dynamic_results, ground_truth))

In [10]:
static_NMIs

[0.5568560422143263,
 0.5496958397847449,
 0.6327759609145253,
 0.7178679648189225,
 0.6537622456376858]

In [11]:
dynamic_NMIs

[0.5313704281871948,
 0.5749838395283821,
 0.6448887874977783,
 0.6832730943226357,
 0.6487106189590918]